<a href="https://colab.research.google.com/github/k-metrics/DAWS2020/blob/master/tidyr%E3%81%A8%E9%9A%A0%E3%81%95%E3%82%8C%E3%81%9F%E4%BE%BF%E5%88%A9%E9%96%A2%E6%95%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# tidyrと隠された便利関数
---



In [ ]:
library(tidyverse)

## 欠損と欠落
分析対象とするデータの一部が失われていることはよくあります。データが失われているというのは

* 観測値の一部がない（セルに抜けがある）
* 観測記録の一部がない（行に抜けがある）

という状態であり、本セッションでは前者を「欠損」、後者を便宜的に「欠落」と呼びます。  

　  
本セッションでは、欠損や欠落を処理する場合に非常に有用な`tidyr`パッケージ関数を紹介します。

### 欠損の例
Rには**欠損**が含まれているサンプルデータセットとして、ニューヨークの大気観測データをまとめた`airquality`データセットがあります。1973年の5月1日から9月30日までの連続した153日間の観測記録が収められています。

In [ ]:
# 表示させると長いので最初と最後だけ
airquality %>% head()
airquality %>% tail()

### 欠落の例
[個票データ](https://www.stat.go.jp/koukou/howto/process/proc3_2.html)を集計したデータにはしばしば欠落があります。例えば新型コロナウィルス検査結果データは、陽性確定者がゼロの日は個票が作成されませんので、個票データの集計結果には陽性確定者ゼロの日が現れません。  

In [ ]:
# このデータは公開データを集計したものです
df <- "https://gist.githubusercontent.com/k-metrics/c963351ca993f84acde7bba38139f046/raw/8d390198a6d9bd9a4c3e5050d86856638d872949/covid19.csv" %>% 
  readr::read_csv() %>%
  dplyr::mutate(n = as.integer(n))
df

このようなデータは、受け取る側が「ゼロだから記録として出てこない」と読み取ってくれるだろうという**暗黙の了解**の元で作成されています。本セッションでは、このような欠落を「暗黙的な欠落」と呼びます。

### 暗黙的な欠落の問題点
暗黙的な欠落はデータ分析において問題を引き起こす場合があります。上記の新型コロナウィルス検査結果の集計データを用いて折線グラフを描いてみましょう。

In [ ]:
# 横軸は日付、縦軸は陽性確定者数です
df %>% 
  ggplot2::ggplot(ggplot2::aes(x = date, y = n)) + 
    ggplot2::geom_line()

### 【質問1】
描いた折線グラフにどのような問題点があるか考えてください。

■挙手または指名で回答■

#### ヒント


In [ ]:
df %>% 
  ggplot2::ggplot(ggplot2::aes(x = date, y = n)) + 
    ggplot2::geom_line() + 
    # 点を打つとデータの有無が分かりやすくなる
    ggplot2::geom_point()

### 【質問2】
質問1の回答で挙げられた問題点を解決するための方法を考えてください（Rの関数名などを出す必要はありません）。  
　  
※ 「正しいグラフが描くためには、どうしたらいいか？」を考えてください。

### 【回答】
■挙手または指名で回答■

### 【回答例】

以下のコードを実行して確認してください。

In [ ]:
# カレンダーを作り
cal_data <- data.frame(date = seq.Date(from = lubridate::as_date("2020/1/15"),
                                       to = lubridate::as_date("2020/2/4"),
                                       by = "day"))
cal_data %>% head()

# カレンダーと対象のデータを結合して
dplyr::left_join(cal_data, df, by = c("date")) %>% 
# 欠損値をゼロに変換する
  dplyr::mutate(n = dplyr::if_else(is.na(n), 0L, n))

## 欠落の変換

`tidyr::complete`関数は暗黙的な欠落を明示的な欠落に変換する関数です。本セッションにおいて「明示的な欠落」とは、「ここ（この日）にはデータがありません」と分かる状態を意味します。  

では、明示的な欠落に変換したデータでグラフを描き、暗示的な欠落のグラフと比較してください。

In [ ]:
df %>% 
  tidyr::complete(date = seq.Date(min(date), max(date), by = "day"),
                  fill = list(n = 0L)) %>% print() %>%
  ggplot2::ggplot(ggplot2::aes(x = date, y = n)) + 
    ggplot2::geom_line() + 
    ggplot2::geom_point()

### 組み合わせデータの変換
次に複数の変量（フィーチャー）の組み合わせを変換するにはどのようにすれば良いでしょうか？  
以下のデータで考えてみましょう。

In [ ]:
x <- data.frame(group = c(1:2, 1), item_id = c(1:2, 2),
                item_name = c("a", "b", "b"),
                value1 = 1:3, value2 = 4:6)
x

### 【質問】
上記のデータは`group`と`item_id`の組み合わせ（4通りの組み合わせ）の一部が欠落しているデータです。このデータの（暗示的な）欠落を明示的な欠落に変換するための手順を考えｍ説明してください（Rの関数名などを出す必要はありません）。


### 【回答】
■挙手または指名で回答■

### 【回答例】

以下のコードを実行して確認してください。

In [ ]:
x %>%
# 考えうる組み合わせを洗い出し
   with(expand.grid(group = unique(group), item_id = unique(item_id))) %>% 
# 洗い出した組み合わせと対象のデータを結合する
   dplyr::left_join(x)
# 欠損値の返還は省略

### 【Tips】

データを良く見ると`item_id`と`item_name`は連動しています。`item_name`も合わせて変換するには`tidyr::nesting`関数を用いてパラメータを指定します。

In [ ]:
x %>% 
  tidyr::complete(group, tidyr::nesting(item_id, item_name))

## 欠損値の変換


### 欠損を一括で変換する
欠損値（`NA`）が特定の値と等価（等値）である場合には、欠損値を一括して変換したいものです。  
　  
このような場合には`tidyr::replace_na`関数を使います。

In [ ]:
data.frame(x = c(1, 2, NA), y = c("a", NA, "b")) %>% 
   print() %>% 
   tidyr::replace_na(list(x = 0, y = "unknown"))

### 規則的な欠損を変換する
Excelでセル結合されたデータを読み込むと規則的に欠損していることがあります。これも暗黙的な了解の元で欠損させているデータだと言えます。  
　  
例えば、総務省が公開している[「都道府県コード及び市町村コード」改正一覧表](https://www.soumu.go.jp/denshijiti/code.html)には規則的な欠損が存在しています。

In [ ]:
# パスから直接読み込むことができないので、一旦、ダウンロードしてから読み込む
file <- tempfile(fileext = ".xls")
"https://www.soumu.go.jp/main_content/000562731.xls" %>%
  download.file(file)

# 今回は列を最小限に絞っておく
city_code <- file %>% 
  readxl::read_excel(col_names = FALSE, skip = 4) %>%
  dplyr::select(5:7, 11:12, 14) %>%
  dplyr::rename(`都道府県名` = ...5, `旧コード` = ...6, `旧市町村名` = ...7, `コード` = ...11, `市町村名` = ...12, `改正事由` = ...14)

city_code %>% head(10)

このような規則的な欠損を変換するには`tdiyr::fill`関数を使います。

In [ ]:
city_code %>%
  tidyr::fill(`都道府県名`, .direction = "down") %>% 
  head(10)

「同左」と「改正事由」の欠損値を変換すると改正後の市町村コード一覧が出来上がり。

In [ ]:
city_code %>%
  tidyr::fill(`都道府県名`) %>% 
  dplyr::mutate(`コード` = dplyr::if_else(`コード` == "同左", `旧コード`, `コード`),
                `市町村名` = dplyr::if_else(`市町村名` == "同左", `旧市町村名`, `市町村名`)) %>%
  dplyr::filter(`市町村名` != "削除") %>%
  tidyr::replace_na(list(`改正事由` = "新設"))

## 欠損値の削除
`NA`を含む行を削除する場合には`tdiyr::drop_na`関数を使います。パラメータの指定方法で結果が変わりますので注意してください。

In [ ]:
airquality %>% head()

In [ ]:
airquality %>% 
   tidyr::drop_na() %>%
   head()

In [ ]:
airquality %>% 
   tidyr::drop_na(Ozone) %>%
   head()

# その他の便利な関数
その他の便利な関数についてはチュートリアルファイルを用意してあります。入手方法や具体的な使い方などは、来月のLT大会の時に紹介します。

本資料のコードの一部は各関数のヘルプに記載されているサンプルコードを用いています。  

---

Enjoy!

CC 4.0 BY-NC-SA, Sampo Suzuki